# Project Initialization

In [1]:
# =============================================================================
# DEVELOPMENT MODE TOGGLE
# =============================================================================
# Set USE_LOCAL_SOURCE based on your setup:
#   True  = Use local source code (for developers editing ras-commander)
#   False = Use pip-installed package (for users)
# =============================================================================

USE_LOCAL_SOURCE = False  # <-- TOGGLE THIS

# -----------------------------------------------------------------------------
if USE_LOCAL_SOURCE:
    import sys
    from pathlib import Path
    local_path = str(Path.cwd().parent)  # Parent of examples/ = repo root
    if local_path not in sys.path:
        sys.path.insert(0, local_path)  # Insert at position 0 = highest priority
    print(f"📁 LOCAL SOURCE MODE: Loading from {local_path}/ras_commander")
else:
    print("📦 PIP PACKAGE MODE: Loading installed ras-commander")

# Import ras-commander
from ras_commander import RasExamples, RasPrj, init_ras_project, ras

# Verify which version loaded
import ras_commander
print(f"✓ Loaded: {ras_commander.__file__}")

📦 PIP PACKAGE MODE: Loading installed ras-commander


2025-12-15 12:08:22 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.


✓ Loaded: C:\GH\ras-commander\ras_commander\__init__.py


## Parameters

Configure these values to customize the notebook for your project.

In [ ]:
# =============================================================================
# PARAMETERS - Edit these to customize the notebook
# =============================================================================
from pathlib import Path

# Project Configuration
PROJECT_NAME = "Muncie"           # Example project to extract
RAS_VERSION = "6.6"               # HEC-RAS version (6.3, 6.5, 6.6, etc.)

# Output Configuration
OUTPUTS_DIR = Path("_outputs") / "101_project_initialization"  # Artifacts saved here

# Create output directory
OUTPUTS_DIR.mkdir(parents=True, exist_ok=True)
print(f"Outputs will be saved to: {OUTPUTS_DIR.absolute()}")


##### Optional Code Cell For Development/Testing Mode (Local Copy)
##### Uncomment and run this cell instead of the pip cell above

```python
# For Development Mode, add the parent directory to the Python path
import os
import sys
from pathlib import Path

current_file = Path(os.getcwd()).resolve()
rascmdr_directory = current_file.parent

# Use insert(0) instead of append() to give highest priority to local version
if str(rascmdr_directory) not in sys.path:
    sys.path.insert(0, str(rascmdr_directory))

print("Loading ras-commander from local dev copy")
from ras_commander import *
```

In [2]:
import os
from pathlib import Path
import pandas as pd
from IPython import display

## RAS Commander: Core Concepts

RAS Commander is a Python library that provides tools for automating HEC-RAS tasks. It's built with several key design principles:

1. **Project-Centric Architecture**: Everything revolves around HEC-RAS projects
2. **Two RAS Object Approaches**:
   - **Global `ras` Object**: A singleton for simple scripts
   - **Custom RAS Objects**: Multiple ras project instances for complex workflows
3. **Comprehensive Project Representation**: Each RAS object includes DataFrames for plans, geometries, flows, and boundaries
4. **Logging**: Built-in logging to track operations and debug issues
5. **HDF Support**: Specialized functions for HDF file access (plan results, geometry, etc.)

Let's explore these concepts in practice.

## Downloading Example HEC-RAS Projects

RAS Commander includes a utility to download and extract example HEC-RAS projects. These are useful for learning and testing:

In [3]:
# Extract specific projects we'll use in this tutorial
# This will download them if not present and extract them to the example_projects folder
extracted_paths = RasExamples.extract_project(["Balde Eagle Creek", "BaldEagleCrkMulti2D", "Muncie"], output_path="example_projects_01_project_initialization")
print(extracted_paths)

2025-12-15 12:08:23 - ras_commander.RasExamples - INFO - Found zip file: C:\GH\ras-commander\examples\Example_Projects_6_6.zip


2025-12-15 12:08:23 - ras_commander.RasExamples - INFO - Loading project data from CSV...


2025-12-15 12:08:23 - ras_commander.RasExamples - INFO - Loaded 68 projects from CSV.


2025-12-15 12:08:23 - ras_commander.RasExamples - INFO - ----- RasExamples Extracting Project -----


2025-12-15 12:08:23 - ras_commander.RasExamples - INFO - Extracting project 'Balde Eagle Creek'


...

2025-12-15 12:08:24 - ras_commander.RasExamples - INFO - Successfully extracted project 'BaldEagleCrkMulti2D' to C:\GH\ras-commander\examples\example_projects_01_project_initialization\BaldEagleCrkMulti2D


2025-12-15 12:08:24 - ras_commander.RasExamples - INFO - ----- RasExamples Extracting Project -----


2025-12-15 12:08:24 - ras_commander.RasExamples - INFO - Extracting project 'Muncie'


2025-12-15 12:08:25 - ras_commander.RasExamples - INFO - Successfully extracted project 'Muncie' to C:\GH\ras-commander\examples\example_projects_01_project_initialization\Muncie


[WindowsPath('C:/GH/ras-commander/examples/example_projects_01_project_initialization/Balde Eagle Creek'), WindowsPath('C:/GH/ras-commander/examples/example_projects_01_project_initialization/BaldEagleCrkMulti2D'), WindowsPath('C:/GH/ras-commander/examples/example_projects_01_project_initialization/Muncie')]


# Get Paths for Extracted Example Projects

In [4]:
# Get the parent directory of the first extracted path as our examples directory
examples_dir = extracted_paths[0].parent
print(f"Examples directory: {examples_dir}")


# Define paths to the extracted projects
bald_eagle_path = examples_dir / "Balde Eagle Creek"
multi_2d_path = examples_dir / "BaldEagleCrkMulti2D"
muncie_path = examples_dir / "Muncie"

# Verify the paths exist
for path in [bald_eagle_path, multi_2d_path, muncie_path]:
    print(f"Path {path} exists: {path.exists()}")

Examples directory: C:\GH\ras-commander\examples\example_projects_01_project_initialization
Path C:\GH\ras-commander\examples\example_projects_01_project_initialization\Balde Eagle Creek exists: True
Path C:\GH\ras-commander\examples\example_projects_01_project_initialization\BaldEagleCrkMulti2D exists: True
Path C:\GH\ras-commander\examples\example_projects_01_project_initialization\Muncie exists: True


## Utility Function to Print RAS Object Data

Let's create a utility function to help us explore the contents of RAS objects:

In [5]:
def print_ras_object_data(ras_obj, project_name):
    """Prints comprehensive information about a RAS object"""
    print(f"\n{project_name} Data:")
    print("=" * 50)
    print(f"Project Name: {ras_obj.get_project_name()}")
    print(f"Project Folder: {ras_obj.project_folder}")
    print(f"PRJ File: {ras_obj.prj_file}")
    print(f"HEC-RAS Executable Path: {ras_obj.ras_exe_path}")
    
    print("\nPlan Files DataFrame (ras.plan_df):")
    with pd.option_context('display.max_columns', None):
        display.display(ras_obj.plan_df)
    
    print("\nSteady Flow Files DataFrame:")
    display.display(ras_obj.flow_df)
    
    print("\nUnsteady Flow Files DataFrame (ras.unsteady_df):")
    display.display(ras_obj.unsteady_df)
    
    print("\nGeometry Files DataFrame (ras.geom_df):")
    display.display(ras_obj.geom_df)
    
    print("\nHDF Entries DataFrame (ras.get_hdf_entries()):")
    display.display(ras_obj.get_hdf_entries())
    
    print("\nBoundary Conditions DataFrame (ras.boundaries_df):")
    display.display(ras_obj.boundaries_df)

## Approach 1: Using the Global `ras` Object

The global `ras` object is a singleton instance that persists throughout your script. It's ideal for simple scripts working with a single project.

Key characteristics:
- It's available as `ras` immediately after import
- It's initialized via `init_ras_project()` without saving the return value
- It provides access to all project data through the global `ras` variable
- It's simple to use but can be problematic in complex scenarios

Let's initialize it with the Bald Eagle Creek project:

In [6]:
# Initialize the global ras object with Bald Eagle Creek project
# Note: This updates the global 'ras' object visible throughout the script
# Parameters:
#   - project_folder: Path to the HEC-RAS project folder (required)
#   - ras_version: HEC-RAS version (e.g. "6.5") or path to Ras.exe (required first time)

init_ras_project(bald_eagle_path, RAS_VERSION)
print(f"The global 'ras' object is now initialized with the {ras.project_name} project")

2025-12-15 12:08:25 - ras_commander.RasMap - INFO - Successfully parsed RASMapper file: C:\GH\ras-commander\examples\example_projects_01_project_initialization\Balde Eagle Creek\BaldEagle.rasmap


The global 'ras' object is now initialized with the BaldEagle project


In [7]:
# Find the .prj file in the Bald Eagle Creek folder
prj_file = list(bald_eagle_path.glob("*.prj"))[0]
print(f"Found .prj file: {prj_file}")
print(f"File name: {prj_file.name}\n")

# Initialize using the .prj file path directly (NEW FEATURE!)
# This works exactly the same as passing the folder path
init_ras_project(prj_file, RAS_VERSION)
print(f"Successfully initialized project using .prj file path!")
print(f"Project name: {ras.project_name}")
print(f"Project folder: {ras.project_folder}")
print(f"PRJ file: {ras.prj_file}")

# Verify both methods produce identical results
print(f"\n✓ Both folder path and .prj file path initialization methods produce the same result!")
print(f"✓ The project folder is automatically extracted from the .prj file's parent directory")

2025-12-15 12:08:25 - ras_commander.RasMap - INFO - Successfully parsed RASMapper file: C:\GH\ras-commander\examples\example_projects_01_project_initialization\Balde Eagle Creek\BaldEagle.rasmap


Found .prj file: C:\GH\ras-commander\examples\example_projects_01_project_initialization\Balde Eagle Creek\BaldEagle.prj
File name: BaldEagle.prj

Successfully initialized project using .prj file path!
Project name: BaldEagle
Project folder: C:\GH\ras-commander\examples\example_projects_01_project_initialization\Balde Eagle Creek
PRJ file: C:\GH\ras-commander\examples\example_projects_01_project_initialization\Balde Eagle Creek\BaldEagle.prj

✓ Both folder path and .prj file path initialization methods produce the same result!
✓ The project folder is automatically extracted from the .prj file's parent directory


### Alternative: Initialize Using .prj File Path

**New Feature**: You can now initialize a project by providing the direct path to a `.prj` file instead of the project folder. This is especially useful when:
- Working with file selection dialogs (which return file paths)
- Using configuration files that store specific .prj file paths
- Working with folders that contain multiple projects
- Building command-line tools that accept file paths

The function automatically:
1. Validates that the file has a `.prj` extension
2. Verifies the file contains "**Proj Title=**" to confirm it's a HEC-RAS project file (not a plan file)
3. Extracts the parent folder and uses it as the project folder
4. Optimizes initialization by passing the .prj file directly (avoids re-searching)

Let's see this in action:

In [8]:
# Explore the global ras object with our utility function
print_ras_object_data(ras, "Global RAS Object (Bald Eagle Creek)")


Global RAS Object (Bald Eagle Creek) Data:
Project Name: BaldEagle
Project Folder: C:\GH\ras-commander\examples\example_projects_01_project_initialization\Balde Eagle Creek
PRJ File: C:\GH\ras-commander\examples\example_projects_01_project_initialization\Balde Eagle Creek\BaldEagle.prj
HEC-RAS Executable Path: C:\Program Files (x86)\HEC\HEC-RAS\6.5\Ras.exe

Plan Files DataFrame (ras.plan_df):


,plan_number,unsteady_number,geometry_number,Plan Title,Program Version,Short Identifier,Simulation Date,Computation Interval,Mapping Interval,Run HTab,Run UNet,Run Sediment,Run PostProcess,Run WQNet,UNET Use Existing IB Tables,UNET D1 Cores,UNET D2 Cores,PS Cores,DSS File,Friction Slope Method,HDF_Results_Path,Geom File,Geom Path,Flow File,Flow Path,full_path,flow_type
0,01,02,01,Unsteady with Bridges and Dam,5.00,UnsteadyFlow,"18FEB1999,0000,24FEB1999,0500",2MIN,1HOUR,1,1,0,1,0,-1,0.0,0.0,None,dss,2,None,01,C:\GH\ras-commander\examples\example_projects_...,02,C:\GH\ras-commander\examples\example_projects_...,C:\GH\ras-commander\examples\example_projects_...,Unsteady
1,02,None,01,Steady Flow Run,NaN,SteadyRun,"02/18/1999,0000,02/24/1999,0500",2MIN,NaN,1,1,NaN,1,NaN,NaN,NaN,NaN,None,dss,1,None,01,C:\GH\ras-commander\examples\example_projects_...,02,C:\GH\ras-commander\examples\example_projects_...,C:\GH\ras-commander\examples\example_projects_...,Steady



Steady Flow Files DataFrame:


,flow_number,full_path,unsteady_number
0,02,C:\GH\ras-commander\examples\example_projects_...,None
1,01,C:\GH\ras-commander\examples\example_projects_...,None



Unsteady Flow Files DataFrame (ras.unsteady_df):


...

,geom_file,geom_number,full_path,hdf_path
0,g01,01,C:\GH\ras-commander\examples\example_projects_...,C:\GH\ras-commander\examples\example_projects_...



HDF Entries DataFrame (ras.get_hdf_entries()):


,plan_number,unsteady_number,geometry_number,Plan Title,Program Version,Short Identifier,Simulation Date,Computation Interval,Mapping Interval,Run HTab,...,PS Cores,DSS File,Friction Slope Method,HDF_Results_Path,Geom File,Geom Path,Flow File,Flow Path,full_path,flow_type



Boundary Conditions DataFrame (ras.boundaries_df):


,unsteady_number,boundary_condition_number,river_reach_name,river_station,storage_area_name,pump_station_name,bc_type,hydrograph_type,Interval,DSS Path,...,Flow Title,Program Version,Use Restart,Precipitation Mode,Wind Mode,Met BC=Precipitation|Mode,Met BC=Evapotranspiration|Mode,Met BC=Precipitation|Expanded View,Met BC=Precipitation|Constant Units,Met BC=Precipitation|Gridded Source
0,02,1,Bald Eagle,Loc Hav,138154.4,,Flow Hydrograph,Flow Hydrograph,1HOUR,,...,Flow Hydrograph 2,6.30,0,Disable,No Wind Forces,None,None,0,mm/hr,DSS
1,02,2,Bald Eagle,Loc Hav,81500,,Gate Opening,None,NaN,NaN,...,Flow Hydrograph 2,6.30,0,Disable,No Wind Forces,None,None,0,mm/hr,DSS
2,02,3,Bald Eagle,Loc Hav,659.942,,Rating Curve,None,NaN,,...,Flow Hydrograph 2,6.30,0,Disable,No Wind Forces,None,None,0,mm/hr,DSS


### Understanding the RAS Object Structure

Each RAS object contains several important components:

1. **Project Metadata**:
   - `project_name`: Name of the HEC-RAS project
   - `project_folder`: Directory containing project files
   - `prj_file`: Path to the main .prj file
   - `ras_exe_path`: Path to the HEC-RAS executable

2. **Project DataFrames**:
   - `plan_df`: Information about all plan files (.p*)
   - `flow_df`: Information about all steady flow files (.f*)
   - `unsteady_df`: Information about all unsteady flow files (.u*)
   - `geom_df`: Information about all geometry files (.g*)
   - `boundaries_df`: Information about all boundary conditions

3. **Methods for Data Access**:
   - `get_plan_entries()`: Get plan file information
   - `get_flow_entries()`: Get flow file information
   - `get_unsteady_entries()`: Get unsteady flow file information 
   - `get_geom_entries()`: Get geometry file information
   - `get_hdf_entries()`: Get HDF file paths for result files
   - `get_boundary_conditions()`: Get boundary condition details

Let's see how to access specific information from these components:

In [9]:
# Get the first plan's details
if not ras.plan_df.empty:
    first_plan = ras.plan_df.iloc[0]
    print(f"First plan number: {first_plan['plan_number']}")
    print(f"Plan path: {first_plan['full_path']}")
    
    # Get the geometry file for this plan
    geom_id = first_plan.get('Geom File', '').replace('g', '')
    if geom_id:
        geom_info = ras.geom_df[ras.geom_df['geom_number'] == geom_id]
        if not geom_info.empty:
            print(f"Geometry file: {geom_info.iloc[0]['full_path']}")
    
    # Get the HDF results file for this plan (if exists)
    if 'HDF_Results_Path' in first_plan and first_plan['HDF_Results_Path']:
        print(f"Results file: {first_plan['HDF_Results_Path']}")
else:
    print("No plans found in the project.")

First plan number: 01
Plan path: C:\GH\ras-commander\examples\example_projects_01_project_initialization\Balde Eagle Creek\BaldEagle.p01
Geometry file: C:\GH\ras-commander\examples\example_projects_01_project_initialization\Balde Eagle Creek\BaldEagle.g01


### Working with Boundary Conditions

Boundary conditions define the inputs and outputs of your model. Let's see how to access boundary condition information:

In [10]:
# View the boundary conditions DataFrame
ras.boundaries_df 

,unsteady_number,boundary_condition_number,river_reach_name,river_station,storage_area_name,pump_station_name,bc_type,hydrograph_type,Interval,DSS Path,...,Flow Title,Program Version,Use Restart,Precipitation Mode,Wind Mode,Met BC=Precipitation|Mode,Met BC=Evapotranspiration|Mode,Met BC=Precipitation|Expanded View,Met BC=Precipitation|Constant Units,Met BC=Precipitation|Gridded Source
0,02,1,Bald Eagle,Loc Hav,138154.4,,Flow Hydrograph,Flow Hydrograph,1HOUR,,...,Flow Hydrograph 2,6.30,0,Disable,No Wind Forces,None,None,0,mm/hr,DSS
1,02,2,Bald Eagle,Loc Hav,81500,,Gate Opening,None,NaN,NaN,...,Flow Hydrograph 2,6.30,0,Disable,No Wind Forces,None,None,0,mm/hr,DSS
2,02,3,Bald Eagle,Loc Hav,659.942,,Rating Curve,None,NaN,,...,Flow Hydrograph 2,6.30,0,Disable,No Wind Forces,None,None,0,mm/hr,DSS


## Approach 2: Using Custom RAS Objects

For more complex scripts or when working with multiple projects, it's better to create and use separate RAS objects. This approach:

- Creates independent RAS objects for each project
- Avoids overwriting the global `ras` object
- Provides clearer separation between projects
- Allows working with multiple projects simultaneously
- Requires saving the return value from `init_ras_project()`

Let's initialize multiple projects with custom RAS objects:

In [11]:
# Initialize multiple project instances with custom RAS objects
# Note: This also updates the global 'ras' object each time, but we'll use the custom instances
# Parameters remain the same as before
multi_2d_project = RasPrj()
init_ras_project(multi_2d_path, RAS_VERSION, ras_object=multi_2d_project)
print(f"\nMulti2D project initialized with its own RAS object")

muncie_project = RasPrj()
init_ras_project(muncie_path, RAS_VERSION, ras_object=muncie_project)
print(f"\nMuncie project initialized with its own RAS object")

# Note that the global 'ras' object now points to the Muncie project
# The global 'ras' object gets overwritten every time a project is initialized ,
print(f"\nGlobal 'ras' object now points to: {ras.project_name} since it was the last one initialized.  Avoid the global object when using multiple projects.")

2025-12-15 12:08:25 - ras_commander.RasMap - INFO - Successfully parsed RASMapper file: C:\GH\ras-commander\examples\example_projects_01_project_initialization\BaldEagleCrkMulti2D\BaldEagleDamBrk.rasmap



Multi2D project initialized with its own RAS object


2025-12-15 12:08:25 - ras_commander.RasMap - INFO - Successfully parsed RASMapper file: C:\GH\ras-commander\examples\example_projects_01_project_initialization\Muncie\Muncie.rasmap



Muncie project initialized with its own RAS object

Global 'ras' object now points to: BaldEagle since it was the last one initialized.  Avoid the global object when using multiple projects.


### Exploring Multiple Projects

Now we have three RAS objects:
- `multi_2d_project`: Our custom object for the Multi2D project
- `muncie_project`: Our custom object for the Muncie project
- `ras`: The global object (which now points to Muncie)

Let's examine the Multi2D project:

In [12]:
display.display(multi_2d_project.plan_df)

,plan_number,unsteady_number,geometry_number,Plan Title,Program Version,Short Identifier,Simulation Date,Computation Interval,Mapping Interval,Run HTab,...,Friction Slope Method,UNET D2 SolverType,UNET D2 Name,HDF_Results_Path,Geom File,Geom Path,Flow File,Flow Path,full_path,flow_type
0,13,07,06,PMF with Multi 2D Areas,5.10,PMF Multi 2D,"01JAN1999,1200,04JAN1999,1200",30SEC,30MIN,1,...,1,Pardiso (Direct),193,None,06,C:\GH\ras-commander\examples\example_projects_...,07,C:\GH\ras-commander\examples\example_projects_...,C:\GH\ras-commander\examples\example_projects_...,Unsteady
1,15,12,08,1d-2D Dambreak Refined Grid,5.10,1D-2D Refined Grid,"01JAN1999,1200,04JAN1999,1200",20SEC,5MIN,1,...,1,NaN,BaldEagleCr,None,08,C:\GH\ras-commander\examples\example_projects_...,12,C:\GH\ras-commander\examples\example_projects_...,C:\GH\ras-commander\examples\example_projects_...,Unsteady
2,17,09,10,2D to 1D No Dam,5.00,2D to 1D No Dam,"01JAN1999,1200,06JAN1999,1200",1MIN,5MIN,1,...,1,NaN,Upstream2D,None,10,C:\GH\ras-commander\examples\example_projects_...,09,C:\GH\ras-commander\examples\example_projects_...,C:\GH\ras-commander\examples\example_projects_...,Unsteady
3,18,10,11,2D to 2D Run,5.00,2D to 2D Run,"01JAN1999,1200,04JAN1999,1200",20SEC,5MIN,1,...,1,NaN,BaldEagleCr,None,11,C:\GH\ras-commander\examples\example_projects_...,10,C:\GH\ras-commander\examples\example_projects_...,C:\GH\ras-commander\examples\example_projects_...,Unsteady
4,19,11,12,SA to 2D Dam Break Run,5.00,SA to 2D Dam Break,"01JAN1999,1200,04JAN1999,1200",20SEC,10MIN,1,...,1,NaN,BaldEagleCr,None,12,C:\GH\ras-commander\examples\example_projects_...,11,C:\GH\ras-commander\examples\example_projects_...,C:\GH\ras-commander\examples\example_projects_...,Unsteady
5,03,13,09,Single 2D Area - Internal Dam Structure,5.04,Single 2D,"01JAN1999,1200,04JAN1999,1200",30SEC,10MIN,1,...,1,NaN,BaldEagleCr,None,09,C:\GH\ras-commander\examples\example_projects_...,13,C:\GH\ras-commander\examples\example_projects_...,C:\GH\ras-commander\examples\example_projects_...,Unsteady
6,04,01,13,SA to 2D Area Conn - 2D Levee Structure,5.00,2D Levee Struc,"01JAN1999,1200,04JAN1999,1200",20SEC,5MIN,1,...,1,NaN,BaldEagleCr,None,13,C:\GH\ras-commander\examples\example_projects_...,01,C:\GH\ras-commander\examples\example_projects_...,C:\GH\ras-commander\examples\example_projects_...,Unsteady
7,02,01,01,SA to Detailed 2D Breach,5.10,SA-2D Det Brch,"01JAN1999,1200,04JAN1999,1200",10SEC,5MIN,1,...,1,Pardiso (Direct),BaldEagleCr,None,01,C:\GH\ras-commander\examples\example_projects_...,01,C:\GH\ras-commander\examples\example_projects_...,C:\GH\ras-commander\examples\example_projects_...,Unsteady
8,01,01,01,SA to Detailed 2D Breach FEQ,5.03,SA-2D Det FEQ,"01JAN1999,1200,04JAN1999,1200",5SEC,5MIN,1,...,1,NaN,BaldEagleCr,None,01,C:\GH\ras-commander\examples\example_projects_...,01,C:\GH\ras-commander\examples\example_projects_...,C:\GH\ras-commander\examples\example_projects_...,Unsteady
9,05,02,03,Single 2D area with Bridges FEQ,5.10,Single 2D Bridges FEQ,"01JAN1999,1200,04JAN1999,1200",5SEC,10MIN,-1,...,1,PARDISO (Direct),BaldEagleCr,None,03,C:\GH\ras-commander\examples\example_projects_...,02,C:\GH\ras-commander\examples\example_projects_...,C:\GH\ras-commander\examples\example_projects_...,Unsteady


In [13]:
# Examine the Multi2D project
print_ras_object_data(multi_2d_project, "Multi2D Project")


Multi2D Project Data:
Project Name: BaldEagleDamBrk
Project Folder: C:\GH\ras-commander\examples\example_projects_01_project_initialization\BaldEagleCrkMulti2D
PRJ File: C:\GH\ras-commander\examples\example_projects_01_project_initialization\BaldEagleCrkMulti2D\BaldEagleDamBrk.prj
HEC-RAS Executable Path: C:\Program Files (x86)\HEC\HEC-RAS\6.5\Ras.exe

Plan Files DataFrame (ras.plan_df):


,plan_number,unsteady_number,geometry_number,Plan Title,Program Version,Short Identifier,Simulation Date,Computation Interval,Mapping Interval,Run HTab,Run UNet,Run Sediment,Run PostProcess,Run WQNet,UNET Use Existing IB Tables,UNET 1D Methodology,UNET D1 Cores,UNET D2 Cores,PS Cores,DSS File,Friction Slope Method,UNET D2 SolverType,UNET D2 Name,HDF_Results_Path,Geom File,Geom Path,Flow File,Flow Path,full_path,flow_type
0,13,07,06,PMF with Multi 2D Areas,5.10,PMF Multi 2D,"01JAN1999,1200,04JAN1999,1200",30SEC,30MIN,1,1,0,1,0,-1,Finite Difference,0,8,None,dss,1,Pardiso (Direct),193,None,06,C:\GH\ras-commander\examples\example_projects_...,07,C:\GH\ras-commander\examples\example_projects_...,C:\GH\ras-commander\examples\example_projects_...,Unsteady
1,15,12,08,1d-2D Dambreak Refined Grid,5.10,1D-2D Refined Grid,"01JAN1999,1200,04JAN1999,1200",20SEC,5MIN,1,1,0,1,0,-1,Finite Difference,0,6,None,dss,1,NaN,BaldEagleCr,None,08,C:\GH\ras-commander\examples\example_projects_...,12,C:\GH\ras-commander\examples\example_projects_...,C:\GH\ras-commander\examples\example_projects_...,Unsteady
2,17,09,10,2D to 1D No Dam,5.00,2D to 1D No Dam,"01JAN1999,1200,06JAN1999,1200",1MIN,5MIN,1,1,0,1,0,-1,NaN,0,0,None,dss,1,NaN,Upstream2D,None,10,C:\GH\ras-commander\examples\example_projects_...,09,C:\GH\ras-commander\examples\example_projects_...,C:\GH\ras-commander\examples\example_projects_...,Unsteady
3,18,10,11,2D to 2D Run,5.00,2D to 2D Run,"01JAN1999,1200,04JAN1999,1200",20SEC,5MIN,1,1,0,1,0,-1,NaN,0,8,None,dss,1,NaN,BaldEagleCr,None,11,C:\GH\ras-commander\examples\example_projects_...,10,C:\GH\ras-commander\examples\example_projects_...,C:\GH\ras-commander\examples\example_projects_...,Unsteady
4,19,11,12,SA to 2D Dam Break Run,5.00,SA to 2D Dam Break,"01JAN1999,1200,04JAN1999,1200",20SEC,10MIN,1,1,0,1,0,-1,NaN,0,0,None,dss,1,NaN,BaldEagleCr,None,12,C:\GH\ras-commander\examples\example_projects_...,11,C:\GH\ras-commander\examples\example_projects_...,C:\GH\ras-commander\examples\example_projects_...,Unsteady
5,03,13,09,Single 2D Area - Internal Dam Structure,5.04,Single 2D,"01JAN1999,1200,04JAN1999,1200",30SEC,10MIN,1,1,0,1,0,-1,Finite Difference,0,10,None,dss,1,NaN,BaldEagleCr,None,09,C:\GH\ras-commander\examples\example_projects_...,13,C:\GH\ras-commander\examples\example_projects_...,C:\GH\ras-commander\examples\example_projects_...,Unsteady
6,04,01,13,SA to 2D Area Conn - 2D Levee Structure,5.00,2D Levee Struc,"01JAN1999,1200,04JAN1999,1200",20SEC,5MIN,1,1,0,1,0,-1,NaN,0,6,None,dss,1,NaN,BaldEagleCr,None,13,C:\GH\ras-commander\examples\example_projects_...,01,C:\GH\ras-commander\examples\example_projects_...,C:\GH\ras-commander\examples\example_projects_...,Unsteady
7,02,01,01,SA to Detailed 2D Breach,5.10,SA-2D Det Brch,"01JAN1999,1200,04JAN1999,1200",10SEC,5MIN,1,1,0,1,0,-1,Finite Difference,0,0,None,dss,1,Pardiso (Direct),BaldEagleCr,None,01,C:\GH\ras-commander\examples\example_projects_...,01,C:\GH\ras-commander\examples\example_projects_...,C:\GH\ras-commander\examples\example_projects_...,Unsteady
8,01,01,01,SA to Detailed 2D Breach FEQ,5.03,SA-2D Det FEQ,"01JAN1999,1200,04JAN1999,1200",5SEC,5MIN,1,1,0,1,0,-1,NaN,0,0,None,dss,1,NaN,BaldEagleCr,None,01,C:\GH\ras-commander\examples\example_projects_...,01,C:\GH\ras-commander\examples\example_projects_...,C:\GH\ras-commander\examples\example_projects_...,Unsteady
9,05,02,03,Single 2D area with Bridges FEQ,5.10,Single 2D Bridges FEQ,"01JAN1999,1200,04JAN1999,1200",5SEC,10MIN,-1,-1,0,-1,0,-1,Finite Difference,0,10,None,dss,1,PARDISO (Direct),BaldEagleCr,None,03,C:\GH\ras-commander\examples\example_projects_...,02,C:\GH\ras-commander\examples\example_projects_...,C:\GH\ras-commander\examples\example_projects_...,Unsteady



Steady Flow Files DataFrame:


""



Unsteady Flow Files DataFrame (ras.unsteady_df):


...

,geom_file,geom_number,full_path,hdf_path
0,g06,06,C:\GH\ras-commander\examples\example_projects_...,C:\GH\ras-commander\examples\example_projects_...
1,g08,08,C:\GH\ras-commander\examples\example_projects_...,C:\GH\ras-commander\examples\example_projects_...
2,g10,10,C:\GH\ras-commander\examples\example_projects_...,C:\GH\ras-commander\examples\example_projects_...
3,g11,11,C:\GH\ras-commander\examples\example_projects_...,C:\GH\ras-commander\examples\example_projects_...
4,g12,12,C:\GH\ras-commander\examples\example_projects_...,C:\GH\ras-commander\examples\example_projects_...
5,g09,09,C:\GH\ras-commander\examples\example_projects_...,C:\GH\ras-commander\examples\example_projects_...
6,g13,13,C:\GH\ras-commander\examples\example_projects_...,C:\GH\ras-commander\examples\example_projects_...
7,g01,01,C:\GH\ras-commander\examples\example_projects_...,C:\GH\ras-commander\examples\example_projects_...
8,g03,03,C:\GH\ras-commander\examples\example_projects_...,C:\GH\ras-commander\examples\example_projects_...
9,g02,02,C:\GH\ras-commander\examples\example_projects_...,C:\GH\ras-commander\examples\example_projects_...



HDF Entries DataFrame (ras.get_hdf_entries()):


,plan_number,unsteady_number,geometry_number,Plan Title,Program Version,Short Identifier,Simulation Date,Computation Interval,Mapping Interval,Run HTab,...,Friction Slope Method,UNET D2 SolverType,UNET D2 Name,HDF_Results_Path,Geom File,Geom Path,Flow File,Flow Path,full_path,flow_type



Boundary Conditions DataFrame (ras.boundaries_df):


,unsteady_number,boundary_condition_number,river_reach_name,river_station,storage_area_name,pump_station_name,bc_type,hydrograph_type,Interval,DSS File,...,Flow Title,Program Version,Use Restart,Precipitation Mode,Wind Mode,Met BC=Precipitation|Mode,Met BC=Evapotranspiration|Mode,Met BC=Precipitation|Expanded View,Met BC=Precipitation|Constant Units,Met BC=Precipitation|Gridded Source
0,07,1,Bald Eagle Cr.,Lock Haven,137520,,Flow Hydrograph,Flow Hydrograph,1HOUR,Bald_Eagle_Creek.dss,...,PMF with Multi 2D Areas,5.00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,07,2,Bald Eagle Cr.,Lock Haven,81454,,Gate Opening,None,NaN,NaN,...,PMF with Multi 2D Areas,5.00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,07,3,Bald Eagle Cr.,Lock Haven,28519,,Lateral Inflow Hydrograph,Lateral Inflow Hydrograph,1HOUR,Bald_Eagle_Creek.dss,...,PMF with Multi 2D Areas,5.00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,07,4,Bald Eagle Cr.,Lock Haven,1,,Lateral Inflow Hydrograph,Lateral Inflow Hydrograph,1HOUR,NaN,...,PMF with Multi 2D Areas,5.00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,07,5,Bald Eagle Cr.,Lock Haven,136948,82303,Uniform Lateral Inflow Hydrograph,Uniform Lateral Inflow Hydrograph,1HOUR,Bald_Eagle_Creek.dss,...,PMF with Multi 2D Areas,5.00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,07,6,Bald Eagle Cr.,Lock Haven,80720,67130,Uniform Lateral Inflow Hydrograph,Uniform Lateral Inflow Hydrograph,1HOUR,Bald_Eagle_Creek.dss,...,PMF with Multi 2D Areas,5.00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,07,7,Bald Eagle Cr.,Lock Haven,76865,,Lateral Inflow Hydrograph,Lateral Inflow Hydrograph,1HOUR,Bald_Eagle_Creek.dss,...,PMF with Multi 2D Areas,5.00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,07,8,Bald Eagle Cr.,Lock Haven,67130,,Lateral Inflow Hydrograph,Lateral Inflow Hydrograph,1HOUR,Bald_Eagle_Creek.dss,...,PMF with Multi 2D Areas,5.00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,07,9,Bald Eagle Cr.,Lock Haven,66041,1,Uniform Lateral Inflow Hydrograph,Uniform Lateral Inflow Hydrograph,1HOUR,Bald_Eagle_Creek.dss,...,PMF with Multi 2D Areas,5.00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,07,10,Bald Eagle Cr.,Lock Haven,-1867,,Normal Depth,None,NaN,NaN,...,PMF with Multi 2D Areas,5.00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# Examine the Muncie project
print_ras_object_data(muncie_project, "Muncie Project")


Muncie Project Data:
Project Name: Muncie
Project Folder: C:\GH\ras-commander\examples\example_projects_01_project_initialization\Muncie
PRJ File: C:\GH\ras-commander\examples\example_projects_01_project_initialization\Muncie\Muncie.prj
HEC-RAS Executable Path: C:\Program Files (x86)\HEC\HEC-RAS\6.5\Ras.exe

Plan Files DataFrame (ras.plan_df):


,plan_number,unsteady_number,geometry_number,Plan Title,Program Version,Short Identifier,Simulation Date,Computation Interval,Mapping Interval,Run HTab,Run UNet,Run Sediment,Run PostProcess,Run WQNet,UNET Use Existing IB Tables,UNET 1D Methodology,UNET D1 Cores,UNET D2 Cores,PS Cores,DSS File,Friction Slope Method,UNET D2 SolverType,UNET D2 Name,HDF_Results_Path,Geom File,Geom Path,Flow File,Flow Path,full_path,flow_type
0,01,01,01,Unsteady Multi 9-SA run,5.00,9-SAs,"02JAN1900,0000,02JAN1900,2400",15SEC,5MIN,1,1,0,1,0,-1,NaN,0,0,None,dss,1,NaN,NaN,None,01,C:\GH\ras-commander\examples\example_projects_...,01,C:\GH\ras-commander\examples\example_projects_...,C:\GH\ras-commander\examples\example_projects_...,Unsteady
1,03,01,02,Unsteady Run with 2D 50ft Grid,5.10,2D 50ft Grid,"02JAN1900,0000,02JAN1900,2400",10SEC,5MIN,-1,-1,0,-1,0,-1,Finite Difference,0,4,None,dss,1,Pardiso (Direct),2D Interior Area,None,02,C:\GH\ras-commander\examples\example_projects_...,01,C:\GH\ras-commander\examples\example_projects_...,C:\GH\ras-commander\examples\example_projects_...,Unsteady
2,04,01,04,Unsteady Run with 2D 50ft User n Value R,5.10,50ft User n Regions,"02JAN1900,0000,02JAN1900,2400",10SEC,5MIN,1,1,0,1,0,-1,Finite Difference,0,6,None,dss,1,Pardiso (Direct),2D Interior Area,None,04,C:\GH\ras-commander\examples\example_projects_...,01,C:\GH\ras-commander\examples\example_projects_...,C:\GH\ras-commander\examples\example_projects_...,Unsteady



Steady Flow Files DataFrame:


,flow_number,full_path,unsteady_number
0,01,C:\GH\ras-commander\examples\example_projects_...,None



Unsteady Flow Files DataFrame (ras.unsteady_df):


...

,geom_file,geom_number,full_path,hdf_path
0,g01,01,C:\GH\ras-commander\examples\example_projects_...,C:\GH\ras-commander\examples\example_projects_...
1,g02,02,C:\GH\ras-commander\examples\example_projects_...,C:\GH\ras-commander\examples\example_projects_...
2,g04,04,C:\GH\ras-commander\examples\example_projects_...,C:\GH\ras-commander\examples\example_projects_...



HDF Entries DataFrame (ras.get_hdf_entries()):


,plan_number,unsteady_number,geometry_number,Plan Title,Program Version,Short Identifier,Simulation Date,Computation Interval,Mapping Interval,Run HTab,...,Friction Slope Method,UNET D2 SolverType,UNET D2 Name,HDF_Results_Path,Geom File,Geom Path,Flow File,Flow Path,full_path,flow_type



Boundary Conditions DataFrame (ras.boundaries_df):


,unsteady_number,boundary_condition_number,river_reach_name,river_station,storage_area_name,pump_station_name,bc_type,hydrograph_type,Interval,DSS Path,...,hydrograph_num_values,hydrograph_values,full_path,Flow Title,Program Version,Use Restart,Precipitation Mode,Wind Mode,Met BC=Precipitation|Expanded View,Met BC=Precipitation|Gridded Source
0,01,1,White,Muncie,15696.24,,Flow Hydrograph,Flow Hydrograph,1HOUR,,...,65,"[13500, 14000, 14500, 15000, 15500, 16000, 165...",C:\GH\ras-commander\examples\example_projects_...,Flow Boundary Conditions,6.30,0,Disable,No Wind Forces,0,DSS
1,01,2,White,Muncie,237.6455,,Normal Depth,None,NaN,NaN,...,0,NaN,C:\GH\ras-commander\examples\example_projects_...,Flow Boundary Conditions,6.30,0,Disable,No Wind Forces,0,DSS


### Comparing Projects

Let's compare some key metrics of the two projects:

In [15]:
# Create a comparison table of the two projects
comparison_data = {
    'Project Name': [multi_2d_project.project_name, muncie_project.project_name],
    'Number of Plans': [len(multi_2d_project.plan_df), len(muncie_project.plan_df)],
    'Number of Geometries': [len(multi_2d_project.geom_df), len(muncie_project.geom_df)],
    'Number of Flow Files': [len(multi_2d_project.flow_df), len(muncie_project.flow_df)],
    'Number of Unsteady Files': [len(multi_2d_project.unsteady_df), len(muncie_project.unsteady_df)],
    'Number of Boundary Conditions': [len(multi_2d_project.boundaries_df) if hasattr(multi_2d_project, 'boundaries_df') else 0, 
                                     len(muncie_project.boundaries_df) if hasattr(muncie_project, 'boundaries_df') else 0],
    'HDF Results Available': [len(multi_2d_project.get_hdf_entries()) > 0, len(muncie_project.get_hdf_entries()) > 0]
}

comparison_df = pd.DataFrame(comparison_data)
display.display(comparison_df)

,Project Name,Number of Plans,Number of Geometries,Number of Flow Files,Number of Unsteady Files,Number of Boundary Conditions,HDF Results Available
0,BaldEagleDamBrk,11,10,0,10,51,False
1,Muncie,3,3,1,1,2,False


## RAS Commander: Best Practices

After exploring both approaches, here are some best practices for using RAS Commander:

1. **Choose Your Approach Based on Complexity**:
   - **Simple Scripts** (one project): Use the global `ras` object
   - **Complex Scripts** (multiple projects): Use custom RAS objects

2. **Be Consistent**:
   - Don't mix global and custom approaches in the same script
   - Use descriptive names for custom RAS objects

3. **Working with Project Files**:
   - Access project files through the RAS object's DataFrames
   - Use helper functions like `get_plan_path()` to resolve paths

4. **Error Handling**:
   - Always check for empty DataFrames before accessing their contents
   - Use the built-in logging to track operations

5. **Performance Considerations**:
   - For large projects, consider using the HDF classes directly
   - Cache results of expensive operations when possible

## Summary of Key Functions

- `init_ras_project(project_folder, ras_version)`: Initialize a RAS project
- `RasExamples().extract_project(project_name)`: Extract example projects
- `RasPrj.get_project_name()`: Get the name of the project
- `RasPrj.get_plan_entries()`: Get plan file information
- `RasPrj.get_flow_entries()`: Get flow file information
- `RasPrj.get_unsteady_entries()`: Get unsteady flow file information
- `RasPrj.get_geom_entries()`: Get geometry file information
- `RasPrj.get_hdf_entries()`: Get HDF result file information
- `RasPrj.get_boundary_conditions()`: Get boundary condition details
- `RasPlan.get_plan_path(plan_number)`: Get the path to a plan file
- `RasPlan.get_geom_path(geom_number)`: Get the path to a geometry file
- `RasPlan.get_flow_path(flow_number)`: Get the path to a flow file
- `RasPlan.get_unsteady_path(unsteady_number)`: Get the path to an unsteady flow file

## Discovering Plan Outputs

After running HEC-RAS plans, you can discover what results are available:

### HDF Result Files
```python
# Get HDF results info from plan_df
ras.plan_df[['plan_number', 'Plan Title', 'HDF_Results_Path']]

# Or use get_hdf_entries() for detailed HDF information
ras.get_hdf_entries()
```

### Available Output Discovery Methods
- **`HdfBase.get_dataset_info(plan_number, group_path)`** - Explore HDF structure
- **`HdfResultsPlan.list_steady_variables(plan_number)`** - List available steady state outputs
- **`RasMap.get_results_folder(plan_number)`** - Find raster outputs folder
- **`RasMap.get_results_raster(plan_number, variable_name)`** - Get specific raster path

These discovery methods help you understand what outputs are available before extracting data.

## Next Steps

Now that you understand the basics of project initialization in RAS Commander, you can explore more advanced topics:

1. Working with HDF files for result analysis
2. Modifying plan, geometry, and flow files
3. Running HEC-RAS simulations
4. Extracting and visualizing results
5. Automating model calibration

These topics are covered in other examples and notebooks in the RAS Commander documentation.